# ProtoVQ-VAE

## Resources

### Papers
Melchiorre, Alessandro B., Navid Rekabsaz, Christian Ganhör, and Markus Schedl. 2022. **“ProtoMF: Prototype-Based Matrix Factorization for Effective and Explainable Recommendations.”** In Sixteenth ACM Conference on Recommender Systems, 246–56. Seattle WA USA: ACM. https://doi.org/10.1145/3523227.3546756.

Oord, Aaron van den, Oriol Vinyals, and Koray Kavukcuoglu. 2018. **“Neural Discrete Representation Learning.”** arXiv. http://arxiv.org/abs/1711.00937.

Liang, Dawen, Rahul G. Krishnan, Matthew D. Hoffman, and Tony Jebara. 2018. **“Variational Autoencoders for Collaborative Filtering.”** arXiv. http://arxiv.org/abs/1802.05814.

Shenbin, Ilya, Anton Alekseev, Elena Tutubalina, Valentin Malykh, and Sergey I. Nikolenko. 2020. **“RecVAE: A New Variational Autoencoder for Top-N Recommendations with Implicit Feedback.”** In Proceedings of the 13th International Conference on Web Search and Data Mining, 528–36. Houston TX USA: ACM. https://doi.org/10.1145/3336191.3371831.

### Code
Recbole Base dataset:                           https://github.com/RUCAIBox/RecSysDatasets/blob/master/conversion_tools/src/base_dataset.py 

LFM2b1monDataset adapted from:                  https://github.com/RUCAIBox/RecSysDatasets/blob/master/conversion_tools/src/extended_dataset.py

Implementation of VQ-VAE:                       https://colab.research.google.com/github/zalandoresearch/pytorch-vq-vae/blob/master/vq-vae.ipynb

### Dataset
Last FM Music Listening Events (2020 Subset):   http://www.cp.jku.at/datasets/LFM-2b/

### General
Thesis writing guide (ML institute):            https://docs.google.com/document/d/1p5d0eykqaw0dfB-L62kPOpdetn8X3CgcJQ3wh7RhPlw/edit#heading=h.8hffm3guomu
NeurIPS 2024 template:                          https://www.overleaf.com/latex/templates/neurips-2024/tpsbbrdqcmsh

### Changelog
v0.1: for-loop used for vector quantization, very inefficient
v0.2: for-loop replaced with matrix-based vector quantization




## Imports

In [1]:
# Google Colab only
#!pip install recbole
#!pip install -U ray
#!pip install wandb
#!jupyter nbextension enable --py widgetsnbextension

import os
import sys
import gc
from logging import getLogger
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from recbole.quick_start import run_recbole
from recbole.model.abstract_recommender import GeneralRecommender
from recbole.utils import InputType, init_logger, init_seed, get_flops, set_color
from recbole.model.init import xavier_normal_initialization
from recbole.trainer import Trainer
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.data.dataloader.general_dataloader import FullSortEvalDataLoader
from recbole.data.transform import construct_transform

In [2]:
# Google Colab only
#from google.colab import drive
#drive.mount('/content/drive')
#config = 'drive/MyDrive/Colab Notebooks/data/protovq-vae/config.yaml'

config = os.getcwd() + '/configs/config.yaml'

## GPU Checks

In [3]:
# Empty GPU cache
gc.collect()
torch.cuda.empty_cache()

# Get GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jun 30 17:25:16 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650        Off |   00000000:01:00.0  On |                  N/A |
| 56%   40C    P5             14W /   75W |     902MiB /   4096MiB |     31%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 16.7 gigabytes of available RAM

Not using a high-RAM runtime


## Data Preprocessing
Transforming raw data into atomic format required for RecBole (see https://recbole.io/docs/user_guide/data/atomic_files.html).

LFM2B-1MON download:        http://www.cp.jku.at/datasets/LFM-2b/

=> Extract to folder in CWD:   /data/lfm2b-1mon/original/

In [5]:
# Prepare LFM2B-1M

def data_to_inter(path:str, user_col:str, item_col:str):
    """Turns data into atomic inter format.
    Args:
        path (str): File path
        user_col (str): Column name of user IDs
        item_col (str): Column name of item IDs
    """
    df = pd.read_csv(path)
    df_inter = df[[user_col, item_col]]

    return df_inter

def factorize_lfm2b_1mon():
    df = pd.read_csv('data/atomic/lfm2b-1mon/lfm2b-1mon.inter', sep='\t')
    df.rename(
        columns={'user_id:token': 'user_id_old:token', 'item_id:token': 'item_id_old:token'},
        inplace=True
    )
    df['user_id:token'] = pd.factorize(df['user_id_old:token'])[0]
    df['item_id:token'] = pd.factorize(df['item_id_old:token'])[0]
    columns_new = ['user_id_old:token', 'item_id_old:token', 'user_id:token', 'item_id:token', 'timestamp:float', 'num_repeat:float']
    df = df[columns_new]
    df.to_csv('data/lfm2b-1mon/atomic/lfm2b-1mon_remapped.inter', index=False, sep='\t')


# Uncomment for data preprocessing
#data = data_to_inter('data/lfm2b-1mon/listening_history.csv', 'user_id:token', 'item_id:token')
#data.to_csv('data/lfm2b-1mon/atomic/test_data.inter', index=False, sep='\t')
#factorize_lfm2b_1mon()


In [6]:
class ProtoVQ_VAE(GeneralRecommender):
    input_type = InputType.POINTWISE

    def __init__(self, config, dataset):
        super(ProtoVQ_VAE, self).__init__(config, dataset)

        # Load dataset info
        self.n_users = dataset.user_num
        self.n_items = dataset.item_num

        # Load config
        self.layers = config['mlp_hidden_size']
        #self.lat_dim = config['latent_dimension']
        #self.lat_split = config['latent_split']

        # latent dimension must be divisible without remainder by latent split
        #assert self.lat_dim % self.lat_split == 0, f'lat_dim ({self.lat_dim}) not divisible without remainder by lat_split ({self.lat_split}).'

        #self.proto_dim = int(self.lat_dim / self.lat_split) # dimension of embedding vectors D
        self.proto_dim = config['proto_dim']
        self.n_proto = config['n_proto']                    # number of vectors in codebook K
        self.topk_proto = config['topk_proto']              # number of most similar prototypes
        self.proto_idx_hist = []
        self.drop_out = config['drop_out']
        #self.anneal_cap = config['anneal_cap']
        #self.total_anneal_steps = config['total_anneal_steps']
        self.commitment_cost = config['commitment_cost']    # beta term in paper, term (3)

        # Load item history
        self.history_item_id, self.history_item_value, _ = dataset.history_item_matrix()
        self.history_item_id = self.history_item_id.to(self.device)
        self.history_item_value = self.history_item_value.to(self.device)

        # Create encoder and decoder
        self.encode_layer_dims = [self.n_items] + self.layers + [self.proto_dim]
        self.decode_layer_dims = [int(self.proto_dim)] + self.encode_layer_dims[::-1][1:]
        self.encoder = self.mlp_layers(self.encode_layer_dims)
        self.decoder = self.mlp_layers(self.decode_layer_dims) 

        # Parameter initialization
        self.apply(xavier_normal_initialization)
        
        # Create prototype embeddings
        self.prototypes = nn.Embedding(self.n_proto, self.proto_dim)
        self.prototypes.weight.data.uniform_(-1/self.n_proto, 1/self.n_proto)

        self.update = 0


    def get_rating_matrix(self, user):
        r"""Get a batch of user's features with the user's id and history interaction matrix.

        Args:
            user (torch.LongTensor): Input tensor that contains user's id, shape: [batch_size, ]

        Returns:
            torch.FloatTensor: The user features of a batch of users, shape: [batch_size, n_items]
        """

        # Construct tensor of shape [batch_size, n_items] using tensor of shape [B, H]
        col_indices = self.history_item_id[user].flatten()
        row_indices = (
            torch.arange(user.shape[0])
            .to(self.device)
            .repeat_interleave(self.history_item_id.shape[1], dim=0)
        )
        rating_matrix = (torch.zeros(1).to(self.device).repeat(user.shape[0], self.n_items))
        rating_matrix.index_put_((row_indices, col_indices), self.history_item_value[user].flatten())

        return rating_matrix


    def mlp_layers(self, layer_dims):
        mlp_modules = []
        for i, (d_in, d_out) in enumerate(zip(layer_dims[:-1], layer_dims[1:])):
            mlp_modules.append(nn.Linear(d_in, d_out))
            if i != len(layer_dims[:-1]) - 1:
                mlp_modules.append(nn.ReLU())
        return nn.Sequential(*mlp_modules)


    def vector_quantizer(self, x):
        x_shape = x.shape

        # Flatten input
        flat_input = x.view(-1, self.proto_dim)

        # Calculate distances
        distances = (torch.sum(flat_input**2, dim=1, keepdim=True)
                    + torch.sum(self.prototypes.weight**2, dim=1)
                    - 2 * torch.matmul(flat_input, self.prototypes.weight.t()))
        #distances = torch.dist(flat_input, self.prototypes.weight)


        # Encoding
        encoding_indices = torch.topk(distances, k=self.topk_proto, largest=False).indices 
        #encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        
        # Add to proto_idx_hist to check frequency of prototype vectors being used
        #self.proto_idx_hist.append(encoding_indices)
        
        # Set index of closest prototype vector to 1
        encodings = torch.zeros(encoding_indices.shape[0], self.n_proto, device=x.device)
        encodings.scatter_(1, encoding_indices, 1)

        # Quantize and unflatten
        quantized = torch.matmul(encodings, self.prototypes.weight).view(x_shape) #/ self.topk_proto

        # Loss - Last part of term (3) in paper
        e_latent_loss = F.mse_loss(quantized.detach(), x)
        q_latent_loss = F.mse_loss(quantized, x.detach())
        vq_loss = q_latent_loss + self.commitment_cost * e_latent_loss

        quantized = x + (quantized - x).detach()
        avg_probs = torch.mean(encodings, dim=0)
        perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))

        return quantized, vq_loss, perplexity, encodings


    def forward(self, x):
        # From MultiVAE
        x = F.normalize(x) # Question: Why?
        x = F.dropout(x, self.drop_out, training=self.training)
        x = self.encoder(x)

        # Split tensor
        #x_split = torch.tensor_split(x, self.lat_split, dim=1)

        #z, vq_loss, perplexity = [], [], []
        #for x_ in x:
        #    quantized_, vq_loss_, perplexity_, _ = self.vector_quantizer(x_)
        #    z.append(self.decoder(quantized_))
        #    vq_loss.append(vq_loss_)
        #    perplexity.append(perplexity_)

        quantized, vq_loss, perplexity, _ = self.vector_quantizer(x)
        z = self.decoder(quantized)
        
        #z = torch.stack(z, dim=0)
        #z = torch.sum(torch.stack(z, dim=0), dim=0)
        vq_loss = torch.mean(vq_loss)
        #vq_loss = torch.stack(vq_loss)
        perplexity = torch.mean(perplexity)
        #perplexity = torch.stack(perplexity)

        return z, vq_loss, perplexity


    def calculate_loss(self, interaction):
        user = interaction[self.USER_ID]
        x = self.get_rating_matrix(user) 

        self.update += 1

        #if self.total_anneal_steps > 0:
        #    anneal = min(self.anneal_cap, 1.0 * self.update / self.total_anneal_steps)
        #else:
        #    anneal = self.anneal_cap

        z, vq_loss, perplexity = self.forward(x)

        #vq_loss *= anneal

        #CE loss
        #recon_error = -(F.log_softmax(x_recon, 1) * x).sum(1).mean()
        recon_error = -(F.log_softmax(z, 1) * x).sum(1).mean()
        del z
        #recon_error = F.cross_entropy(x_recon, x)
        #loss = reScon_error + vq_loss

        return recon_error, vq_loss


    def predict(self, interaction):
        user = interaction[self.USER_ID]
        item = interaction[self.ITEM_ID]

        rating_matrix = self.get_rating_matrix(user)

        scores, _, _ = self.forward(rating_matrix)

        return scores[[torch.arange(len(item)).to(self.device), item]]


    def full_sort_predict(self, interaction):
        user = interaction[self.USER_ID]

        rating_matrix = self.get_rating_matrix(user)

        scores, _, _ = self.forward(rating_matrix)

        return scores.view(-1)

In [7]:
def run_recbole(
    model=None, dataset=None, config_file_list=None, config_dict=None, saved=True
):
    r"""A fast running api, which includes the complete process of
    training and testing a model on a specified dataset

    Args:
        model (str, optional): Model name. Defaults to ``None``.
        dataset (str, optional): Dataset name. Defaults to ``None``.
        config_file_list (list, optional): Config files used to modify experiment parameters. Defaults to ``None``.
        config_dict (dict, optional): Parameters dictionary used to modify experiment parameters. Defaults to ``None``.
        saved (bool, optional): Whether to save the model. Defaults to ``True``.
    """
    # configurations initialization
    config = Config(
        model=model,
        dataset=dataset,
        config_file_list=config_file_list,
        config_dict=config_dict,
    )
    init_seed(config["seed"], config["reproducibility"])
    # logger initialization
    init_logger(config)
    logger = getLogger()
    logger.info(sys.argv)
    logger.info(config)

    # dataset filtering
    dataset = create_dataset(config)
    logger.info(dataset)

    # dataset splitting
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # model loading and initialization
    init_seed(config["seed"] + config["local_rank"], config["reproducibility"])
    model = model(config, train_data.dataset).to(config['device'])
    logger.info(model)

    transform = construct_transform(config)
    flops = get_flops(model, dataset, config["device"], logger, transform)
    logger.info(set_color("FLOPs", "blue") + f": {flops}")

    # trainer loading and initialization
    trainer = Trainer(config, model)

    # model training
    best_valid_score, best_valid_result = trainer.fit(
        train_data, valid_data, saved=saved, show_progress=config["show_progress"]
    )

    # Show distribution of prototype indices chosen during all epochs
    print(pd.DataFrame(np.bincount(torch.cat(model.proto_idx_hist).flatten().cpu().numpy())))

    # model evaluation
    test_result = trainer.evaluate(
        test_data, load_best_model=saved, show_progress=config["show_progress"]
    )

    logger.info(set_color("best valid ", "yellow") + f": {best_valid_result}")
    logger.info(set_color("test result", "yellow") + f": {test_result}")

    return {
        "best_valid_score": best_valid_score,
        "valid_score_bigger": config["valid_metric_bigger"],
        "best_valid_result": best_valid_result,
        "test_result": test_result,
    }

In [ ]:
run_recbole(model=ProtoVQ_VAE, config_file_list=[config])